# Simple Index Demo

#### Load documents, build the GPTSimpleVectorIndex

In [1]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from gpt_index import GPTSimpleVectorIndex, SimpleDirectoryReader
from IPython.display import Markdown, display

/Users/jerryliu/Programming/gpt_index/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# load documents
documents = SimpleDirectoryReader('../paul_graham_essay/data').load_data()

In [3]:
index = GPTSimpleVectorIndex.from_documents(documents)

INFO:gpt_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:gpt_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 17617 tokens
> [build_index_from_nodes] Total embedding token usage: 17617 tokens


In [4]:
# save index to disk
index.save_to_disk('index_simple.json')

In [5]:
# load index from disk
index = GPTSimpleVectorIndex.load_from_disk('index_simple.json')

#### Query Index

In [ ]:
# set Logging to DEBUG for more detailed outputs
response = index.query("What did the author do growing up?")

In [ ]:
display(Markdown(f"<b>{response}</b>"))

**Query Index with custom embedding string**

In [ ]:
from gpt_index.indices.query.schema import QueryBundle

In [ ]:
query_bundle = QueryBundle(
    query_str="What did the author do growing up?", 
    custom_embedding_strs=['The author grew up painting.']
)
response = index.query(query_bundle)

In [ ]:
display(Markdown(f"<b>{response}</b>"))

#### Get Sources

In [ ]:
print(response.get_formatted_sources())

#### Query Index with LlamaLogger

Log intermediate outputs and view/use them.

In [6]:
from gpt_index.logger import LlamaLogger
from gpt_index import ServiceContext

llama_logger = LlamaLogger()
service_context = ServiceContext.from_defaults(llama_logger=llama_logger)

In [ ]:
response = index.query(
    "What did the author do growing up?",
    service_context=service_context,
    similarity_top_k=2,
    # response_mode="tree_summarize"
)

In [ ]:
# get logs
service_context.llama_logger.get_logs()